## Chain 鏈條
- Primitive 基元: llm, prompt template, 文檔加載器 
- Chain 鏈條: 由Primitive 組成

In [1]:
import os 
import logging 

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
api_key = os.environ.get('OPENAI_API_KEY')
if api_key is None:
    raise ValueError("The OPENAI_API_KEY environment variable is not set.")

In [2]:
import os 
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(
    openai_api_base=os.environ["CHATGPT_API_ENDPOINT"],
    openai_api_key=os.environ["OPENAI_API_KEY"])

/Users/babyching/Documents/personal/LLM/從LangChain接入ChatGPT到製作股票分析AI團隊/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Utility Chain 工具鏈

In [3]:
template = '''
    回答如下問題, 並說出準確的計算答案
    問題: {query}
    AI: 
'''

In [4]:
from langchain import PromptTemplate 
prompt_template = PromptTemplate(
    input_variables = ["query"],
    template = template
)

In [5]:
from langchain.chains.llm import LLMChain

chain = LLMChain(
    llm=chat,
    prompt=prompt_template,
    verbose = True 
)

In [6]:
print(18 ** 0.6352)

6.271177528595222


In [7]:
output = chain.invoke({"query": "18的0.6352次方是多少"})
print(output["text"])



> Entering new LLMChain chain...
Prompt after formatting:

    回答如下問題, 並說出準確的計算答案
    問題: 18的0.6352次方是多少
    AI: 


> Finished chain.
18的0.6352次方是 5.6891。


In [8]:
from langchain.chains import LLMMathChain
math_chain = LLMMathChain.from_llm(
    llm=chat,
    verbose = True
)


In [9]:
result = math_chain.invoke({"question": "18的0.6352次方是多少"})



> Entering new LLMMathChain chain...
18的0.6352次方是多少```text
18**0.6352
```
...numexpr.evaluate("18**0.6352")...

Answer: 6.271177528595222
> Finished chain.


In [12]:
print(math_chain.prompt.template)


Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



### Sequential Chain

In [15]:
import re 

def transform_func(inputs:dict):
    text = inputs["tr_text"]
    # Remove extra spaces
    text = re.sub(r'[ ]+', ' ', text)

    return {"tr_output": text}

In [17]:
from langchain.chains.transform import TransformChain
clean_spaces_chain = TransformChain(
    input_variables=["tr_text"],
    output_variables=["tr_output"],
    transform=transform_func
)


In [19]:
output = clean_spaces_chain.invoke({"tr_text":"             Hello         Worlds!~      "})
print(output["tr_output"])

 Hello Worlds!~ 


In [22]:
template = '''
  轉換如下文字: 
  {cvr_output}

  新形式{cvr_style}
  
  轉換: 
'''

prompt = PromptTemplate(
    input_variables = ["cvr_output","cvr_style"]
    template = template
)

In [24]:
convert_chain = LLMChain(
    llm=chat,
    prompt = prompt,
    output_key = "new_output",
    verbose = True
)


In [27]:
new_output = convert_chain.invoke({"cvr_output":"How are you", "cvr_style":"變中文"})
print(new_output["new_output"])



> Entering new LLMChain chain...
Prompt after formatting:

  轉換如下文字: 
  How are you

  新形式變中文
  
  轉換: 


> Finished chain.
你好嗎？


### 連接chain

In [29]:
import re 

def transform_func(inputs:dict):
    text = inputs["tr_text"]
    # Remove extra spaces
    text = re.sub(r'[ ]+', ' ', text)

    return {"tr_output": text}

from langchain.chains.transform import TransformChain
clean_spaces_chain = TransformChain(
    input_variables=["tr_text"],
    output_variables=["tr_output"],
    transform=transform_func
)

template = '''
  轉換如下文字: 
  {tr_output}

  新形式{cvr_style}
  
  轉換: 
'''

prompt = PromptTemplate(
    # input_variables = ["cvr_output","cvr_style"]
    input_variables = ["cvr_style"],
    template = template
)

convert_chain = LLMChain(
    llm=chat,
    prompt = prompt,
    output_key = "new_output",
    verbose = True
)


In [31]:
from langchain.chains import SequentialChain

sequential_chain = SequentialChain(
    chains = [clean_spaces_chain, convert_chain],
    input_variables = ["tr_text", "cvr_style"],
    output_variables = ["new_output"]
)

In [40]:
input_text = '''
"Gong Hey Fat Choy"               is a traditional               phrase              used to convey well-wishes during          the      New Year       celebration.
'''

In [41]:
output = sequential_chain.invoke({"tr_text": input_text, "cvr_style": "轉換成中文" })
print(output["new_output"])



> Entering new LLMChain chain...
Prompt after formatting:

  轉換如下文字: 
  
"Gong Hey Fat Choy" is a traditional phrase used to convey well-wishes during the New Year celebration.


  新形式轉換成中文
  
  轉換: 


> Finished chain.
"恭喜發財"是一個傳統短語，用於在新年慶祝期間表達祝福。
